In [23]:
#подключение необходимых библиотек
import numpy as np
from matplotlib import  pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation
import time

In [24]:
#Возвращает значение полинома вида kind степени n в точке x
def polynomial (n, x, kind):
    #моном
    if (kind == 'monom'):
        return x**n
    #Полином Чебышева
    elif (kind == 'Chebyshev'):
        if(n == 0) :
            return 1
        elif (n == 1) :
            return x
        else:
            return 2*x*polynomial(n-1, x, kind) - polynomial(n -2, x, kind)
    #Полином Лежандра
    elif (kind == 'Legendre'):
        if (n == 0):
            return 1
        elif (n == 1):
            return x
        else:
            return (2*n -1) / n*x*polynomial(n-1, x, kind) - (n-1) / n * polynomial(n-2, x, kind)

#Функция составления матрицы A и вектора правой части b
def set_matrix (n, data, kind) :
    A = np.zeros((len(data), n+1))
    b = np.zeros(len(data))
    
    for i in range (len(data)) :
        for j in range (n+1):
            A[i][j] = polynomial(j, data[i][0], kind)
        b[i] = data [i][1]
    return A, b

#Функция аппроксимации данных с помощью QR алгоритма
def data_approximation_QR(n, data, kind) :
    start_time = time.time()

    A, b = set_matrix (n, data, kind)

    #todo Householder
    #QR разложение матрицы A
    Q, R = np.linalg.qr(A)

    #Выводим число обусловленности матрицы
    #print ('cond(A): ', np.linalg.cond(A))
    cond_value = np.linalg.cond(R)

    #Решение переопределенной СЛАУ QR методом
    coeffs = np.dot(np.linalg.inv(R), np.dot(Q.T, b))
    #Подсчет SME
    SME = np.sqrt(np.sum((np.dot(A, coeffs) - b)**2) / len(data)) / np.max(np.abs(b))
    #print ('SME: ', SME)
    
    f_approx = np.dot(A, coeffs)
    SME = np.sqrt(np.sum((f_approx - b)**2) / len(data)) / np.max(np.abs(b))
    
    end_time = time.time()
    computation_time = end_time - start_time
    return cond_value, SME, f_approx, coeffs, computation_time

#Функция аппроксимации данных методом НУ
def data_approximation_NU(n, data, type):
    start_time = time.time()
     
    A, b = set_matrix(n, data, type)
    #Формируем матрицу AtA и вектор Atb
    AtA = np.dot(A.T, A)
    np.savetxt("Ata.txt", AtA)
    Atb = np.dot(A.T, b)

    #Вычисляем число обусловленности матрицы
    cond_value = np.linalg.cond(AtA)
    
    #Выводим число обусловленности матрицы
    #print ('cond(AtA): ', cond_value)

    #Решение СЛАУ
    coeffs = np.linalg.solve(AtA, Atb)
    #Подсчет SME
    SME = np.sqrt(np.sum((np.dot(A, coeffs) - b)**2) / len(data)) / np.max(np.abs(b))
    #print ('SME: ', SME)

    f_approx = np.dot(A, coeffs)

    end_time = time.time()
    computation_time = end_time - start_time
    return cond_value, SME, f_approx, coeffs, computation_time
    #return np.dot(A, coeffs)

from matplotlib import  pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation

#функция для построения графиков: data - исходные данные (x, y), approximation_data - данные численного алгоритма (f(x), y)
def plot_graphics(data, approximation_data, n):
    fig = plt.figure(figsize=(12,8), frameon=True)
    plt.style.use('ggplot')
    #plt.rcParams["mathtext.fontset"] = "cm"
    #plt.rcParams["font.family"] = "Times New Roman"
    plt.rcParams['font.size'] = 37
    plt.rcParams['text.color'] = 'black'
    plt.rcParams['xtick.color'] = 'black'
    plt.rcParams['ytick.color'] = 'black'
    plt.rcParams['axes.labelcolor'] = 'black'
    ax = fig.add_subplot(111)    

    ax.spines['bottom'].set_color('black')
    ax.spines['top'].set_color('black')
    ax.spines['right'].set_color('black')
    ax.spines['left'].set_color('black')

    ax.set(facecolor='w')
    ax.grid('axis = "both"', color = 'gray')

    ax.set_xlabel('$x$', labelpad = -10)
    ax.set_ylabel('$y$', rotation = 0, labelpad = 20)

    ax.plot(data[:,0], data[:,1], color = 'blue', linestyle = '-', linewidth = 3, label='Данные')
    #ax.plot(w_obr, P_obr, color = 'red', linestyle = '-', label = 'obrabotannoe')
    ax.plot(data[:,0], approximation_data, color = 'red', linestyle = '-', linewidth = 2, label = 'Приближение при N = ' + str(n))
    ax.legend(loc=4)

    plt.show()

In [25]:
#Пример вызова функции аппроксимации данных
File_name = 'data/data_1.txt'
data = np.loadtxt(File_name, dtype = float)
#Степень полинома
max_n = 10


#запись всех данных в файл, чтобы оттуда записать в таблицу =(
# Открываем файл для записи результатов
with open('comparison_simple.txt', 'w', encoding='utf-8') as f:
    
    f.write("СРАВНЕНИЕ МЕТОДОВ АППРОКСИМАЦИИ\n")
    f.write("Файл данных: " + File_name + "\n")
    f.write("Максимальная степень полинома: " + str(max_n ) + "\n\n")
    
    f.write("МЕТОД НОРМАЛЬНЫХ УРАВНЕНИЙ (НУ):\n")
    for max_n  in range(max_n  + 1):
        try:
            cond_value, SME, f_approx, coeffs, comp_time = data_approximation_NU(max_n , data, 'monom')
            f.write(f"n={max_n }: {cond_value:.6e} {SME:.6e} {comp_time:.6f} сек\n")
        except np.linalg.LinAlgError:
            f.write(f"n={max_n }: Ошибка - вырожденная матрица\n")
    
    f.write("\nМЕТОД QR-РАЗЛОЖЕНИЯ:\n")
    for max_n  in range(max_n  + 1):
        try:
            cond_value, SME, f_approx, coeffs, comp_time = data_approximation_QR(max_n , data, 'monom')
            f.write(f"n={max_n }: {cond_value:.6e} {SME:.6e} {comp_time:.6f} сек\n")
        except np.linalg.LinAlgError:
            f.write(f"n={max_n }: Ошибка - вырожденная матрица\n")
    
    # Простое сравнение методов
    f.write("\nСРАВНЕНИЕ МЕТОДОВ:\n")
    
    # Собираем данные для сравнения
    nu_results = []
    qr_results = []
    
    for max_n  in range(max_n  + 1):
        try:
            cond_nu, sme_nu, _, _, time_nu = data_approximation_NU(max_n , data, 'monom')
            nu_results.append((max_n , cond_nu, sme_nu, time_nu))
        except:
            nu_results.append((max_n , None, None, None))
            
        try:
            cond_qr, sme_qr, _, _, time_qr = data_approximation_QR(max_n , data, 'monom')
            qr_results.append((max_n , cond_qr, sme_qr, time_qr))
        except:
            qr_results.append((max_n , None, None, None))
    
    # Находим лучшие результаты
    valid_nu = [r for r in nu_results if r[2] is not None]
    valid_qr = [r for r in qr_results if r[2] is not None]
    
    if valid_nu:
        best_nu = min(valid_nu, key=lambda x: x[2])  # минимальная SME
        f.write(f"Лучший результат НУ: n={best_nu[0]}, SME={best_nu[2]:.6e}\n")
    
    if valid_qr:
        best_qr = min(valid_qr, key=lambda x: x[2])  # минимальная SME
        f.write(f"Лучший результат QR: n={best_qr[0]}, SME={best_qr[2]:.6e}\n")
    
    # Сравнение времени
    times_nu = [r[3] for r in valid_nu if r[3] is not None]
    times_qr = [r[3] for r in valid_qr if r[3] is not None]
    
    if times_nu and times_qr:
        avg_time_nu = sum(times_nu) / len(times_nu)
        avg_time_qr = sum(times_qr) / len(times_qr)
        f.write(f"Среднее время НУ: {avg_time_nu:.6f} сек\n")
        f.write(f"Среднее время QR: {avg_time_qr:.6f} сек\n")

print("Результаты сохранены в файл: comparison_simple.txt")


#for n in range (max_n):
    
    # print ('n: ', n)
    #Вызов функции
    # print ('n: ', n)
    #c = data_approximation_NU(n, data, 'monom')
    #print ('test1 \t test2 \t test3 \t test4')
    #строим графики
    # plot_graphics(data, c, n)

    
# #Вызов функции
# c = data_approximation_NU(n, data, 'monom')
# #строим графики
# plot_graphics(data, c, n)

Результаты сохранены в файл: comparison_simple.txt
